In [1]:
#@Imports

#############################
# region: Imports and Setup #
#############################
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload
#############################
# endregion                 #
#############################

import datetime
from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function
from concordia.components import agent as components

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb import plan as tpb_plan
from examples.tpb import agent as tpb_agent

In [2]:
SETUP_TIME=dt(year=2024,month=10,day=1,hour=18)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [3]:
config = tpb_agent.AgentConfig(
  name="Alice",
  gender="female",
  context="Alice loves to cook, but doesn't know what a Space Shuttle is.",
  goal="Alice wants to become a patissier.",
  traits="Determined, scatterbrained, picky",
  date_of_birth=dt(year=1993,month=11,day=1,hour=00),
  extras={
    'model': model,
    'embedder': embedder,  
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

As a young girl, Alice was already showing signs of her determined nature. She would spend hours in the kitchen with her grandmother, watching as she whipped up delicious meals and treats. Alice's scatterbrained tendencies were also evident, as she would often get distracted by a particularly interesting bug or a stray thread on her favorite stuffed animal. Despite this, she was always eager to learn and help out, whether it was setting the table or helping with meal prep.

As Alice grew older, her love of cooking only intensified. She began experimenting with new recipes and ingredients, much to the delight (and occasionally dismay) of her family. Her pickiness became more pronounced as well, as she developed a keen sense of taste and smell that allowed her to detect even the slightest imperfections in a dish. Despite this, she remained determined to become a professional chef, pouring over cookbooks and watching cooking shows with rapt attention.

As Alice entered her golden years, s

In [4]:
params = {
  "model": model,
  "config": config,
  "num_memories": 100,
  "clock_now": clock.now,
  "verbose": True,
  "num_behavs": 5,
  "num_people": 5,
  "threading": False
}

# Memory
memory = tpb_memory.EpisodicMemory(**params)

# Situation
situation = components.situation_perception.SituationPerception(
  "situation",
  model=model,
  memory=config.memory,
  agent_name=config.name,
  clock_now = clock.now,
  num_memories_to_retrieve=100,
  verbose=True
)

# TPB pipeline components
behav = tpb_components.Behaviour("behaviour", **params)
attitude = tpb_components.Attitude("attitude", components=[behav], **params)
people = tpb_components.People("people", components=[behav], **params)
motiv = tpb_components.Motivation("motivation", components=[people], **params)
norm = tpb_components.SubjectiveNorm("norm", components=[motiv], **params)
control = tpb_components.BehaviouralControl("control", components=[behav], **params)
thin_goal = tpb_components.ThinGoal("thin_goal", **params)
intention = tpb_components.BehaviouralIntention(
  "intention", 
  components=[attitude, norm, control, thin_goal], 
  **params
)
plan = tpb_plan.TPBPlan(
  "plan",
  goal = intention,
  components=[situation],
  **params
)

full_model = tpb_components.TPBModel(
  "full_model",
  components=[
    memory,
    behav,
    attitude,
    people,
    motiv,
    norm,
    control,
    thin_goal,
    situation,
    intention,
    plan
  ]
)

In [5]:
full_model.update()

[LOG] [Component] behaviour initialized.
[LOG] [Component] attitude initialized.
[LOG] [Component] people initialized.
[LOG] [Component] motivation initialized.
[LOG] [Component] control initialized.
[LOG] [Component] intention initialized.
[LOG] [Component] thin_goal initialized.
As Alice deliberated, she recognized that her behaviors could lead to feelings of guilt and regret among those she cared about if she chose not to take action, potentially causing a rift in relationships and impacting her sense of self-worth.I think I can continue the story from here!

Alice is currently 25 years old...

Please let me know if I'm correct!Memories of Alice:
[01 Nov 1999 00:00:00] When Alice was 6 years old, she spent hours playing with her grandmother's old wooden spoon collection, pretending each one was a magical wand. She would stir imaginary pots of soup and make "mmm" noises as she tasted the air. This marked the beginning of her love affair with cooking.
[01 Nov 2002 00:00:00] As Alice t

In [6]:
full_model.update()

Based on Alice's traits and goals, here are five potential behaviors she could exhibit:

1. **Dive into cooking research**: Alice could spend hours researching different patisserie techniques, recipes, and ingredients to improve her skills and stay ahead of the game.
2. **Seek guidance from Chef Emma**: As a childhood idol and instructor, Chef Emma's expertise would be invaluable in helping Alice refine her patisserie skills and achieve her goal.
3. **Experiment with new recipes**: With a passion for cooking and determination to become a patissier, Alice might decide to try out new recipes and techniques to challenge herself and expand her culinary repertoire.
4. **Take online courses or attend workshops**: To further her education and stay current in the field, Alice could enroll in online courses or attend workshops focused on patisserie, pastry-making, or baking.
5. **Start a food blog or YouTube channel**: As a way to share her passion for cooking and build a community around her g

In [7]:
%autoreload 2
from examples.tpb.agent import TPBAgent
from concordia.typing import agent as simulacrum_agent

CALL_TO_ACTION = (
    "Given the above, what will {agent_name} do for the "
    "next {timedelta} to achieve their current plan?"
  )

agent = TPBAgent(
  config=config,
  full_model=full_model,
  clock=clock
)

action = agent.act(
  action_spec=simulacrum_agent.ActionSpec(call_to_action = CALL_TO_ACTION, output_type = "FREE")
)

print(action)

Finish cooking dinner.Finish cooking dinner.


In [21]:
%autoreload 2
from examples.tpb.utils import pprint
from concordia.document import interactive_document
from concordia.thought_chains import thought_chains

cot = interactive_document.InteractiveDocument(model=model)
mems = '\n'.join(config.memory.retrieve_associative(
  query=intention.state(),
  k=5
))

cot.statement(f"Relevant memories of {agent.name}: {mems}")
cot.statement(f"Situation of {agent.name}: {situation.state()}")
cot.statement(f"Attempted action of {agent.name}: {intention.state()}")
cot.statement(f"{agent.name} is performing this action for the next hour.")
result = thought_chains.attempt_to_result(
  cot,
  action,
  agent.name
)
print('\n\n#####\n\n')
print(result)
print('\n\n#####\n\n')

I'm happy to help!

As Alice continues her goal to experiment with new recipes, she decides to try out a recipe for lemon bars that she found online. She heads to her kitchen, where she has all the necessary ingredients and tools.

As she begins to mix together the crust, filling, and topping, she realizes that the recipe is a bit more complex than she initially thought. But she's determined to get it right and decides to take her time to make sure everything turns out perfectly.

After about 45 minutes of cooking and baking, Alice takes the lemon bars out of the oven and lets them cool for a few minutes before slicing into them. She can't help but feel a sense of accomplishment as she takes her first bite - the combination of tangy lemon filling and sweet shortbread crust is absolutely divine!

As she savors the flavors, Alice starts thinking about what other recipes she wants to try out next. She pulls up her phone and starts browsing through food blogs and recipe websites, looking f

In [11]:
utils.save_component(
  f"./saved_states/full_model_{dt.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl",
  full_model
)

utils.save_component(
  f"./saved_states/intention_{dt.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl",
  intention
)

utils.save_memories(
  f"./saved_states/memory_{dt.now().strftime('%Y-%m-%d_%H-%M-%S')}.pkl",
  config.memory
)

Here is Alice's revised plan for the rest of the day:

[18:00 - 19:00] Start experimenting in the kitchen by trying out a new recipe. She can choose something simple like making a batch of cookies or attempting to make a cake.

[19:00 - 20:00] Take a break and relax with some TV time. This time, she can watch a cooking show or a documentary about space exploration (since she doesn't know what a Space Shuttle is, this might be an interesting topic for her).

[20:00 - 21:00] Continue experimenting in the kitchen by trying out a new technique or ingredient. She can also take some time to review her plans and goals for opening her own bakery.

[21:00 - 22:00] Spend some time reading cookbooks or online recipes to get inspiration for new dishes and techniques. This will help her develop her skills and stay motivated towards achieving her goal.

[22:00 - 23:00] Take some time to reflect on her day and think about what she's grateful for, like the opportunity to pursue her passion for baking.

In [1]:
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

from datetime import datetime as dt
from datetime import timedelta as td

from concordia.associative_memory import importance_function
from concordia.components import agent as components

from examples.custom_components.setup import model_setup, measurement_setup
from examples.tpb import utils
from examples.tpb import components as tpb_components
from examples.tpb import memory as tpb_memory
from examples.tpb import plan as tpb_plan
from examples.tpb import survey as tpb_survey
from examples.tpb import agent as tpb_agent

SETUP_TIME=dt(year=2015,month=10,day=1,hour=18)
TIME_STEP=td(minutes=5)

model, embedder = model_setup("llama3")
measurement, clock = measurement_setup(setup_time=SETUP_TIME,time_step=TIME_STEP)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [2]:
config = tpb_agent.AgentConfig(
  name="Alice",
  gender="female",
  context="Alice loves to cook, but doesn't know what a Space Shuttle is.",
  goal="Alice wants to become a patissier.",
  traits="Determined, scatterbrained, picky",
  date_of_birth=dt(year=1993,month=11,day=1,hour=00),
  extras={
    'model': model,
    'embedder': embedder,  
    'importance': importance_model.importance,
    'clock_now': clock.now
    }
)

config.memory.add(
  text="Alice is a student at the local university",
  timestamp=SETUP_TIME - td(days=7)
)
config.memory.add(
  text="Alice sees a link sent out on her university listserv inviting students to participate in a research study.",
  timestamp=SETUP_TIME - td(hours=1)
)
config.memory.add(
  text="Alice clicks the link and begins the study.",
  timestamp=SETUP_TIME - td(minutes=5)
)

As a young girl, Alice was determined to help her mother in the kitchen, often getting underfoot as she tried to "assist" with meal prep. Her scatterbrained nature meant that utensils and ingredients would get lost in the chaos, but her pickiness ensured that every dish that left the kitchen was a masterpiece. As she grew older, Alice's love for cooking only intensified, and she began to dream of opening her own bakery.

As an adult, Alice's days were filled with the sweet scent of freshly baked bread wafting from her cozy shop on Main Street. Her customers adored her, not just for her delectable treats but also for her kind heart and willingness to go the extra mile. Despite her busy schedule, Alice always made time for her family and friends, often hosting dinner parties that left everyone stuffed and happy. But beneath her warm exterior, Alice had a secret fear: failure. She worried that one day, her bakery wouldn't be enough, and she'd have to find a new passion to pursue.

As the 

In [3]:
%autoreload 2
from examples.tpb import survey as tpb_survey
from concordia.language_model import ollama_model


model_mc = ollama_model.OllamaProbs(
  "llama3:70b"
)

survey = tpb_survey.Survey(
  model=model_mc,
  config=config,
  clock_now=clock.now,
  interventions=(
    "attitude",
    "norm",
    "control"
  )
)

survey.update()

In [4]:
survey.answers

[{'attitude': {'awareness': False, 'importance': '4'}},
 {'norm': {'awareness': False, 'importance': '4'}},
 {'control': {'awareness': False, 'importance': '3'}}]

In [ ]:



# from concordia.components import game_master as gm_components
# from concordia.associative_memory import associative_memory
# gm_memory = associative_memory.AssociativeMemory(
#   embedder=embedder,
#   importance=importance_model_gm.importance,
#   clock=clock.now,
# )

# effect = gm_components.direct_effect.DirectEffect(
#   players = agent,
#   clock_now=clock.now,
#   model=model,
#   memory=gm_memory,
#   verbose=True
# )



